# Project 4: Wikipedia Semantic Search

*Author: William Buck*

### Summary of download.py, search.py, and predict.py files
The three .py files that are run in this notebook will do the following:
1. Download all of the page text, pageids, and titles from pages from a certain category.
    - This information is stored in a mongoDB set up on an AWS instance. The client connection is set in all three of the scripts as ```MongoClient('35.163.182.105', 27016)```
    - The way I have organized the data in mongo: 
        - Each category that is downloaded has its own collection in the mongo database. 
        - A collection is also made for every subcategory of the orginally downloaded category if nesting_level is specified.
        - Each document in a collection is a page that falls under the category in Wikipedia.
1. Search for any word or phrase in the contents of the downloaded Wikipedia pages.
    - A DataFrame must be created in the Data folder of the downloaded information to search.
1. Predict the category of a page from the wikipdia page title.
    - predict.py analyzes all of the page content related to a certain category, and when a page title is searched, it uses the Wikipedia API to get all of the page text of the passed title, then predicts the category of the page based on that content.
    - The downloaded data must be stored in a pandas DataFrame in order for categorical predictions to be made.

**Note about TruncatedSVD:**
<br>TruncatedSVD is included as an option before finding cosine similarity in both the predict.py and search.py scripts. It makes the results much slower, and it also is not always completely accurate. I think that this is becuase I'm not using an ```ngram_range``` in the TFIDF transformation. However, I believe that doing this would increase its accuracy, but it keeps causing the kernel of this notebook to crash because the DataFrame is too big.


## Example of download.py 

### Getting all of the page contents for the categories specified once the download.py file is run.

A nesting level must be set due to Wikipedia's infinite nesting structure. It is impossible to get all of the subcategories using a recursive function because often, there is a child category that includes its parent category as a subcategory, which creates a loop.

**Overwrite will delete the database in mongo if it has been previously created.**

After downloading, it will ask if you would like to merge and pickle all of the collections in the mongo database. This needs to be done if you want to search or predict on the downloaded categories later on.

In [1]:
%run download.py

Database name: animals
Category to download: Weasels
Nesting level: 0
Overwrite? [y/n] n

Starting download of category "Weasels"
Finished updating "animals" with all page content from category "Weasels".

There were no subcategory pages included for "Weasels".

Continue downloading? [y/n] y
Category to download: bears
Nesting level: 1

Starting download of category "bears"
Finished updating "animals" with all page content from category "bears".

There are 29 subcategories that need to be added to animals.

28.27.26.25.24.23.22.21.20.19.18.17.16.15.14.13.12.11.10.9.8.7.6.5.4.3.2.1.0.
Continue downloading? [y/n] y
Category to download: dolphins
Nesting level: 0

Starting download of category "dolphins"
Finished updating "animals" with all page content from category "dolphins".

There were no subcategory pages included for "dolphins".

Continue downloading? [y/n] n
Merge all downloads to DataFrame and pickle? [y/n] y
Completed merge.


In [13]:
animals_df = pd.read_pickle('Data/animals_download_merged_df.p')
animals_df.sample(10)

,_id,category,content,page_id,title
577,5a24ad2a830fdb056f227284,Polar bears,The Agreement on the Conservation of Polar Bea...,26621962,Agreement on the Conservation of Polar Bears
8,5a249ea8830fdb0514906ff3,extinct reptiles,Eastwoods longtailed seps or Eastwoods whip li...,12505208,Eastwood's long-tailed seps
1142,5a24ad06830fdb056f22724f,Individual giant pandas,Mei Sheng Chinese 美生 lit Beautiful Life or Bor...,12306593,Mei Sheng
42,5a24ab57830fdb056f226ffd,Weasels,The Amazon weasel Mustela africana also known ...,2431523,Amazon weasel
303,5a24ac40830fdb056f22712e,Fictional bears,,54238684,Fluffy and Uranus
418,5a24a354830fdb051490714c,snakes,A snake farm is a facility that houses and bre...,6487602,Snake farm
334,5a24ac54830fdb056f22714d,Fictional bears,Paw Paws also known as Paw Paw Bears is an Ame...,4104102,Paw Paws
524,5a24ac70830fdb056f227178,Fictional pandas,Hello Panda is a brand of Japanese biscuit man...,7372998,Hello Panda
1004,5a24a010830fdb0514907042,birds,A fecal sac also spelled faecal sac is a mucou...,9986783,Fecal sac
1236,5a24ac9c830fdb056f2271b5,Films about bears,Care Bears Movie II A New Generation is a 1986...,1569551,Care Bears Movie II: A New Generation


## Example of search.py

I have included several searches for random phrases, or phrases copied from one of the downloaded Wikipedia articles stored in mongoDB. I have included the same search with both TruncatedSVD and without so that you can see the difference.

In [4]:
%run search.py

Pickle file to search: merged_wiki_databases_df.p
Use TruncatedSVD with search? [y/n] y
Search: steve fredette


,cosine_sim
title,
T/Maker,0.558983
Sapphire (comics),0.539630
Josiah Power,0.530929
ConveyIQ,0.530686
Holborn 9100,0.525728


Make another search? [y/n] y
Use TruncatedSVD with search? [y/n] n
Search: steve fredette


,cosine_sim
title,
"Toast, Inc.",0.189185
NeXTMail,0.098005
Excellence (software),0.068193
Google Kythe,0.063147
PaperClip,0.057697


Make another search? [y/n] y
Use TruncatedSVD with search? [y/n] y
Search: singletrack mountain wheel carbon fiber


...

Make another search? [y/n] y
Use TruncatedSVD with search? [y/n] n
Search: cell organism evolution nature neural network


,cosine_sim
title,
Random neural network,0.353669
NeuroSolutions,0.328901
Neural network software,0.327340
Category:Neural network software,0.326828
Computational neurogenetic modeling,0.317758


Make another search? [y/n] y
Use TruncatedSVD with search? [y/n] y
Search: surly bikes


,cosine_sim
title,
Polygon Bikes,0.971181
Tommaso bikes,0.956306
Novara (company),0.955968
Kent International,0.952718
Ridley Bikes,0.936826


Make another search? [y/n] n


In [9]:
%run search.py

Pickle file to search: animals_download_merged_df.p
Use TruncatedSVD with search? [y/n] y
Search: polar bear


,cosine_sim
title,
The Polar Bear Family & Me,0.917822
International Polar Bear Day,0.906216
Agreement on the Conservation of Polar Bears,0.867391
Ursus maritimus tyrannus,0.850274
Polar bear,0.844850


Make another search? [y/n] y
Use TruncatedSVD with search? [y/n] n
Search: polar bear


,cosine_sim
title,
Polar bear,0.758262
Agreement on the Conservation of Polar Bears,0.648881
Grizzly–polar bear hybrid,0.584722
The Polar Bear Family & Me,0.584233
International Polar Bear Day,0.513278


Make another search? [y/n] y
Use TruncatedSVD with search? [y/n] y
Search: rattlesnake


...

Make another search? [y/n] y
Use TruncatedSVD with search? [y/n] y
Search: thumbs hands intelligent


,cosine_sim
title,
Hand (unit),0.813766
Pony,0.720979
Glossary of equestrian terms,0.716803
Hippology,0.699486
Horse theft,0.693968


Make another search? [y/n] y
Use TruncatedSVD with search? [y/n] n
Search: thumbs hands intelligent


,cosine_sim
title,
Hand (unit),0.164814
Pony,0.067853
Noogenesis,0.052628
Denise L. Herzing,0.051113
Bear (comics),0.038893


Make another search? [y/n] n


### Predict category from page title

The 5 items returned are the predictions for page Category based on a page title. If the page is not an actual title, then an error will be returned.

I have included several searches for page titles of downloaded Wikipedia articles stored in mongoDB. I've also included the same search with both TruncatedSVD and without so that you can see the difference.

In [8]:
%run predict.py

Pickle file to search: animals_download_merged_df.p
Use TruncatedSVD to predict? [y/n] y
Page Title: dinosaur


,cosine_sim
category,
Dinosaurs,0.986166
birds,0.213328
extinct reptiles,0.173561
humans,0.139263
apes,0.077978


Make another category prediction from page title? [y/n] y
Use TruncatedSVD to predict? [y/n] n
Page Title: dinosaur


,cosine_sim
category,
Dinosaurs,0.887742
birds,0.192037
extinct reptiles,0.156239
humans,0.125364
apes,0.070195


Make another category prediction from page title? [y/n] y
Use TruncatedSVD to predict? [y/n] y
Page Title: Glacier bear


...

Make another category prediction from page title? [y/n] y
Use TruncatedSVD to predict? [y/n] n
Page Title: Felinology


,cosine_sim
category,
cats,0.240667
humans,0.029758
dogs,0.025799
birds,0.017702
Giant pandas,0.015363


Make another category prediction from page title? [y/n] y
Use TruncatedSVD to predict? [y/n] n
Page Title: Operation Cat Drop


,cosine_sim
category,
cats,0.416098
Fictional bears,0.036147
humans,0.033430
dogs,0.031603
Films about bears,0.030424


Make another category prediction from page title? [y/n] n


In [9]:
%run predict.py

Pickle file to search: merged_wiki_databases_df.p
Use TruncatedSVD to predict? [y/n] y
Page Title: machine learning


,cosine_sim
category,
Deep learning,0.878219
Machine learning researchers,0.774625
Artificial neural networks,0.773371
Neural network software,0.762730
Structured prediction,0.732735


Make another category prediction from page title? [y/n] y
Use TruncatedSVD to predict? [y/n] n
Page Title: machine learning


,cosine_sim
category,
Deep learning,0.510043
machine learning,0.452217
Machine learning researchers,0.433527
Data mining and machine learning software,0.414052
Structured prediction,0.365086


Make another category prediction from page title? [y/n] y
Use TruncatedSVD to predict? [y/n] y
Page Title: Gore Creek Trail


...

Make another category prediction from page title? [y/n] y
Use TruncatedSVD to predict? [y/n] n
Page Title: Tom Brown Jr.


,cosine_sim
category,
Franklin Dam,0.232312
wilderness,0.183369
IUCN Category Ib,0.112497
Astrobiologists,0.104187
Wilderness Areas of the United States,0.103926


Make another category prediction from page title? [y/n] y
Use TruncatedSVD to predict? [y/n] y
Page Title: Tom Brown Jr.


,cosine_sim
category,
wilderness,0.724330
Franklin Dam,0.637318
Wilderness medicine,0.603228
Wilderness Areas of the United States,0.585490
Wilderness areas,0.582430


Make another category prediction from page title? [y/n] n
